# **Multi-Agent Research System**
This project is a multi-agent system (MAS) built to fulfill the coding task for the RCCIIT Internship shortlisting. The system uses the LangGraph framework  to create a chain of specialized AI agents that work together to answer complex research queries.


The architecture is based on the "Proposed Sample MAS Architecture" diagram provided in the task.


# **🏛️Project Architecture**
The system is a graph composed of five distinct agents, each with a specific role:

### *Query Planner Agent:*
This agent receives the user's query. It uses Tree of Thought (ToT) prompting to decompose the query into smaller, atomic questions and determines the best retrieval strategy (internal vector search, external web search, or both).

### *Retriever Agent:*
This agent conditionally fetches information based on the planner's strategy.

It retrieves internal documents from a ChromaDB vector store.

It accesses external information using the Tavily web search tool.

### *Synthesizer Agent:*
This agent takes all the retrieved context (both internal and external) and "integrates, aligns, filters, and compares" it to synthesize a comprehensive draft answer.

### *Reviewer Agent:*
This agent acts as a quality control step. It validates the factual consistency of the synthesized answer against the retrieved context and computes a faithfulness score.

### *Writer Agent:*
The final agent takes the reviewed answer and formats it into a structured, professional report in Markdown, including citations and the confidence score.

# **🚀Tech Stack**
This project was built in Google Colab  and utilizes the following technologies:

**Graph Framework:** LangGraph


**LLM (Open Source):**  Llama 3.1 (via the Groq API)


**Vector Database:** ChromaDB

**Embedding Model:** HuggingFace "all-MiniLM-L6-v2" (runs locally)


**Web Search Tool:** Tavily AI

Core Libraries: LangChain, Python


**Chunking Strategy:** The RAG pipeline demonstrates two chunking strategies (RecursiveCharacterTextSplitter and SentenceTransformersTokenTextSplitter) as required.

# **⚙️Setup and Installation**
Environment: This notebook is designed to be run in Google Colab.

Install Dependencies: The first cell in the notebook installs all required Python packages.

#### **Bash--**
*[ !pip install -q langgraph langchain langchain_community chromadb tavily-python sentence-transformers langchain-groq langchain-tavily ]*


####**API Keys:**
 When you run the second cell, you will be prompted to enter two API keys:

**GROQ_API_KEY:** For the Llama 3.1 model.

**TAVILY_API_KEY:** For the web search tool.

# **▶️How to Run**
1. Open the notebook in Google Colab.

2. Run all cells from top to bottom, starting with the installation cell.

3. The final cell will execute the agent graph.

4. You will be prompted to provide your query, like this:

**Please enter your research query:** *[Your query here]*

**Example Query : Tell me the process of how a Machine learning model can differentiate between different patterns with the help of neural networks ?**

The notebook will then print the step-by-step execution of each agent and display the final, formatted report.

In [1]:
import os
import json
from typing import TypedDict, List, Optional
from dotenv import load_dotenv, find_dotenv  # <-- 1. IMPORT find_dotenv

# --- LangChain Core Imports ---
from langchain_core.documents import Document
from langchain_core.messages import AIMessage
# ... (all your other imports) ...
from langchain_community.document_loaders import PyMuPDFLoader, UnstructuredFileLoader
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, END
from IPython.display import display, Markdown

# --- Load API Keys from .env file ---
load_dotenv(find_dotenv())  # <-- 2. USE find_dotenv() HERE

# Check if keys are loaded
if not os.getenv("GROQ_API_KEY") or not os.getenv("TAVILY_API_KEY"):
    print("ERROR: API keys not found. Make sure your .env file is in the project root.")
else:
    print("--- API keys loaded successfully from .env file ---")

print("--- Imports and API key setup complete. ---")

d:\Google Colab\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- API keys loaded successfully from .env file ---
--- Imports and API key setup complete. ---


# RAG SETUP & CHUNKING STRATEGIES

In [2]:
import os
import json
from dotenv import load_dotenv  # <-- Now this import will work
from typing import TypedDict, List, Optional

# --- LangChain Core Imports ---
from langchain_core.documents import Document
from langchain_core.messages import AIMessage

# --- LangChain Community Imports ---
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader, UnstructuredFileLoader

# --- LangChain Partner Package Imports ---
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch

# --- LangGraph Imports ---
from langgraph.graph import StateGraph, END

# --- Standard Library Imports ---
from IPython.display import display, Markdown

# --- 1. Load API Keys from .env file ---
load_dotenv() 
print("--- .env file loaded ---")

# --- 2. Initialize Models and Tools (FIXED) ---
print("--- Initializing LLM and Tools... ---")

# Manually get keys from environment
groq_key = os.getenv("GROQ_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

# Check if keys were actually loaded
if not groq_key:
    raise ValueError("GROQ_API_KEY not found. Check your .env file.")
if not tavily_key:
    raise ValueError("TAVILY_API_KEY not found. Check your .env file.")

# Pass keys directly as parameters to fix API key errors
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0, groq_api_key=groq_key)
web_search_tool = TavilySearch(k=3, tavily_api_key=tavily_key)

print("--- LLM and Tools initialized ---")

# --- 3. Define Helper Function ---
def extract_json_from_response(text: str) -> str:
    """
    Extracts the JSON blob from an LLM response.
    """
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1:
        raise ValueError(f"Could not find JSON object in LLM response: {text}")
    return text[start:end+1]

# --- 4. RAG Setup ---
print("--- Setting up RAG... ---")
doc_dir = "internal_docs/" # Use a relative path for VS Code
os.makedirs(doc_dir, exist_ok=True)
print(f"Created directory: {doc_dir}")
print("="*80)
print(f"Make sure your files (.pdf, .txt, .doc, .docx) are in the '{doc_dir}' folder.")
print("="*80)

all_documents = []
loaded_files = os.listdir(doc_dir)

if not loaded_files:
    print("No files found. Please add files to 'internal_docs' and re-run.")
    retriever = None
else:
    print(f"Found {len(loaded_files)} files. Loading...")
    for file_name in loaded_files:
        file_path = os.path.join(doc_dir, file_name)
        try:
            if file_name.endswith(".pdf"):
                loader = PyMuPDFLoader(file_path)
                all_documents.extend(loader.load())
            elif file_name.endswith((".txt", ".doc", ".docx")):
                loader = UnstructuredFileLoader(file_path)
                all_documents.extend(loader.load())
        except Exception as e:
            print(f"Error loading {file_name}: {e}")

    print(f"\n--- Total documents loaded: {len(all_documents)} ---")
    
    # --- 5. Chunking Strategies ---
    print("\n--- Demonstrating Chunking Strategies... ---")
    recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks_recursive = recursive_splitter.split_documents(all_documents)
    print(f"Strategy 1 (Recursive) produced {len(chunks_recursive)} chunks.")

    token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=20, tokens_per_chunk=100)
    chunks_token = token_splitter.split_documents(all_documents)
    print(f"Strategy 2 (Token-based) produced {len(chunks_token)} chunks.")

    # --- Strategy 3: Agentic Chunking ---
    print("\n--- Applying Strategy 3 (Agentic Chunking)... ---")
    def agentic_chunker(document, llm_model):
        prompt = f"""You are a data scientist... (Your full chunking prompt) ..."""
        try:
            response = llm_model.invoke(prompt)
            json_string = extract_json_from_response(response.content)
            data = json.loads(json_string)
            agent_chunks = [Document(page_content=chunk["content"], metadata={"source": document.metadata.get("source", "unknown"), "agentic_topic": chunk["topic_name"]}) for chunk in data.get("chunks", [])]
            return agent_chunks
        except Exception as e:
            print(f"  - Agentic chunking failed for a document: {e}")
            return [document]

    chunks_agentic = []
    # Process only the first 3 docs to save time
    for doc in all_documents[:3]: 
        chunks_agentic.extend(agentic_chunker(doc, llm)) 
    print(f"Strategy 3 (Agentic) produced {len(chunks_agentic)} chunks.")

    # --- 6. Create Vector Store ---
    print("\n--- Initializing HuggingFace Embeddings... ---")
    embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(documents=chunks_recursive, embedding=embedding_function)
    retriever = vectorstore.as_retriever()
    print("--- Vector Store Created Successfully. ---")

--- .env file loaded ---
--- Initializing LLM and Tools... ---
--- LLM and Tools initialized ---
--- Setting up RAG... ---
Created directory: internal_docs/
Make sure your files (.pdf, .txt, .doc, .docx) are in the 'internal_docs/' folder.
Found 3 files. Loading...

--- Total documents loaded: 903 ---

--- Demonstrating Chunking Strategies... ---
Strategy 1 (Recursive) produced 4782 chunks.
Strategy 2 (Token-based) produced 6652 chunks.

--- Applying Strategy 3 (Agentic Chunking)... ---
  - Agentic chunking failed for a document: Could not find JSON object in LLM response: I'm a data scientist with a passion for extracting insights from complex data sets. I have a strong background in machine learning, statistics, and programming, with expertise in languages such as Python, R, and SQL.

My day typically starts with reviewing the latest data sets and research papers in my field. I'm always on the lookout for new techniques and methodologies that can help me improve my models and make mo

C:\Users\dibya\AppData\Local\Temp\ipykernel_25844\989768449.py:123: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


--- Vector Store Created Successfully. ---


# DEFINE AGENTS AND TOOLS

In [6]:
import json
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch

print("--- Defining Agents and Tools... ---")

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
web_search_tool = TavilySearch(k=3)


def extract_json_from_response(text: str) -> str:
    """
    Extracts the JSON blob from an LLM response that might include
    markdown backticks.
    """
    start = text.find('{')
    end = text.rfind('}')

    if start == -1 or end == -1:
        raise ValueError(f"Could not find JSON object in LLM response: {text}")

    return text[start:end+1]

# --- 1. Query Planner Agent ---
def query_planner_agent(state):
    """
    Decomposes the user query and determines the retrieval strategy
    using Tree of Thought (ToT) prompting.
    """
    print("--- 🧠 Executing Query Planner ---")

    prompt = f"""You are a meticulous and strategic research planner. Your role is to analyze a complex user query and break it down into a logical, step-by-step execution plan for a team of agents.

    **User Query:** "{state['original_query']}"

    **Your Task:**
    1.  **Decompose:** Analyze the query and break it down into smaller, self-contained, atomic questions that must be answered to fulfill the user's request.
    2.  **Strategize:** For each atomic question, decide the best retrieval source. Use 'internal_vector_search' for questions about proprietary data, internal documents, or specific in-house knowledge. Use 'web_search' for general knowledge, definitions, market comparisons, or public information. If a query requires both, use 'both'.
    3.  **Synthesize:** Consolidate this plan into a single, valid JSON object.

    **Output Format:**
    You MUST provide only the final JSON object as your response. Do not add any text, markdown, or commentary before or after the JSON block.
    The JSON object must have two keys:
    1.  `decomposed_queries`: A list of **strings**, where each string is an atomic query.
    2.  `retrieval_strategy`: A single string: 'internal_vector_search', 'web_search', or 'both'.

    **Example:**
    `{{"decomposed_queries": ["What is the in-house DL framework?", "Who are the top 3 market leaders in deep learning?", "Benchmark performance of in-house vs market leaders"], "retrieval_strategy": "both"}}`
    """

    response = llm.invoke(prompt)

    try:
        json_string = extract_json_from_response(response.content)
        plan = json.loads(json_string)
    except Exception as e:
        print(f"Error parsing JSON from Planner: {e}")
        print(f"LLM Response was: {response.content}")
        raise

    return {"decomposed_queries": plan['decomposed_queries'], "retrieval_strategy": plan['retrieval_strategy']}

# --- 2. Retriever Agent ---
def retriever_agent(state):
    """
    Retrieves information based on the planner's strategy.
    """
    print(f"--- 🔍 Executing Retriever (Strategy: {state['retrieval_strategy']}) ---")
    context = []

    query_list = state['decomposed_queries']

    for q in query_list:
        query_string = ""
        if isinstance(q, dict):
            if 'query' in q:
                query_string = q['query']
            elif 'question' in q:
                query_string = q['question']
            elif q.values():
                query_string = str(list(q.values())[0])
        elif isinstance(q, str):
            query_string = q

        if not query_string:
            continue

        if state['retrieval_strategy'] in ['internal_vector_search', 'both']:
            if retriever: # Check if retriever was successfully created
                docs = retriever.invoke(query_string)
                context.extend([f"Internal Doc: {d.page_content}" for d in docs])
            else:
                print("Warning: Internal search requested, but retriever is not available.")

        if state['retrieval_strategy'] in ['web_search', 'both']:
            search_results = web_search_tool.invoke(query_string)
            context.extend([f"Web Result: {res}" for res in search_results])

    return {"retrieved_context": context}

# --- 3. Synthesizer Agent ---
def synthesizer_agent(state):
    """
    Integrates and synthesizes the retrieved context into a draft answer.
    """
    print("--- ✍️ Executing Synthesizer ---")
    prompt = f"""You are a critical research analyst and synthesizer. Your goal is not to just list information, but to **weave a comprehensive, neutral, and coherent narrative** from the provided context.

    **Original Query:** "{state['original_query']}"
    
    **Retrieved Context (from internal docs and web search):**
    ---
    {state['retrieved_context']}
    ---

    **Your Task:**
    1.  Read the Original Query to understand the user's core intent.
    2.  Thoroughly read all pieces of Retrieved Context.
    3.  **Critically analyze** the context. If you find both 'Internal Doc' and 'Web Result' sources, you **must** compare them.
    4.  Write a single, flowing draft answer that directly addresses the Original Query.
    5.  **Important:** When information conflicts (e.g., an internal doc says one thing, a web result says another), you must **explicitly point out this discrepancy** in your draft.
    6.  **Do not add any information or make any assumptions** that are not present in the Retrieved Context. Your answer must be 100% grounded in the provided sources.

    **Draft Answer:**
    """

    response = llm.invoke(prompt)
    return {"synthesized_answer": response.content}

# --- 4. Reviewer Agent ---
def reviewer_agent(state):
    """
    Validates the synthesized answer for factual consistency and
    computes a faithfulness score.
    """
    print("--- ✅ Executing Reviewer ---")
    prompt = f"""You are a meticulous fact-checking and quality assurance editor. Your sole purpose is to ensure the `synthesized_answer` is 100% faithful to the `retrieved_context` and fully complete.

    **Original Query:** "{state['original_query']}"

    **Retrieved Context:**
    ---
    {state['retrieved_context']}
    ---
    
    **Synthesized Answer (Draft):**
    ---
    {state['synthesized_answer']}
    ---

    **Your Task:**
    1.  **Faithfulness Check:** Read the `synthesized_answer` sentence by sentence. Verify that *every single claim* is directly supported by a fact in the `retrieved_context`.
    2.  **Completeness Check:** Ensure the draft answer addresses all parts of the `original_query`.
    3.  **Provide a Score:** Give a `faithfulness_score` from 0.0 (total fabrication) to 1.0 (perfectly supported).
    4.  **Provide Notes:** Briefly explain your score. If the score is less than 1.0, you **must** flag the specific claims that were unfaithful or fabricated.

    **Output Format:**
    You MUST return a single, valid JSON object. Do not add any text before or after it.
    The JSON object **must** have a key named `faithfulness_score` (a float) and a key named `review_notes` (a string).
    
    **Example:**
    `{{"faithfulness_score": 0.8, "review_notes": "The answer is mostly faithful, but the claim about 'v3.0' is not supported by the context."}}`
    """
    response = llm.invoke(prompt)

    try:
        json_string = extract_json_from_response(response.content)
        review = json.loads(json_string)

        print(f"Reviewer LLM Response (Parsed): {review}")

        score = 0.0 # Default
        if 'faithfulness_score' in review:
            score = review['faithfulness_score']
        elif 'score' in review:
            score = review['score']
        elif 'faithfulness' in review:
            score = review['faithfulness']
        else:
            print("Warning: Could not find 'faithfulness_score' or 'score' in reviewer output. Defaulting to 0.0")

    except Exception as e:
        print(f"Error parsing JSON from Reviewer: {e}")
        print(f"LLM Response was: {response.content}")
        raise

    return {"reviewed_answer": state['synthesized_answer'], "faithfulness_score": score}

# --- 5. Writer Agent (IMPROVED MARKDOWN) ---
def writer_agent(state):
    """
    Generates the final, structured report with citations and confidence.
    """
    print("--- 📄 Executing Writer ---")
    prompt = f"""You are a **senior technical writer** preparing a final report for an executive briefing. Your writing style is **formal, objective, clear, and concise**.

    **Reviewed Answer:** "{state['reviewed_answer']}"
    **Faithfulness Score:** {state['faithfulness_score']}

    **Your Task:**
    Transform the `reviewed_answer` into a polished, professional report.
    
    The report **must** use the following Markdown structure:
    
    # [Create a Clear, Descriptive Title for the Report]
    
    ##  Executive Summary
    (Write a 1-2 sentence summary of the main answer here.)
    
    ---
    
    ## Detailed Findings
    (Format the main body of the `reviewed_answer` here. Use `###` H3 headings for sub-topics and bullet points for lists to make it scannable and easy to read.)
    
    ---
    
    ## Confidence & Sources
    
    ### Confidence Score
    **{state['faithfulness_score']*100:.0f}%**
    
    ### Sources
    (Cite the information from the 'Internal Doc' and 'Web Result' bold labels found in the reviewed answer. Format as a bulleted list.)
    ### References
    (Cite all references used in the report here in a bulleted list.)
    **Final Report (in Markdown format):**
    """
    response = llm.invoke(prompt)
    return {"final_report": response.content}

print("--- All agents defined successfully. ---")

--- Defining Agents and Tools... ---
--- All agents defined successfully. ---


# ASSEMBLE THE LANGGRAPH WORKFLOW

In [7]:
print("--- Assembling LangGraph workflow... ---")

class GraphState(TypedDict):
    original_query: str
    decomposed_queries: List[str]
    retrieval_strategy: str
    retrieved_context: List[str]
    synthesized_answer: str
    reviewed_answer: str
    faithfulness_score: Optional[float]
    final_report: str

workflow = StateGraph(GraphState)

workflow.add_node("planner", query_planner_agent)
workflow.add_node("retriever", retriever_agent)
workflow.add_node("synthesizer", synthesizer_agent)
workflow.add_node("reviewer", reviewer_agent)
workflow.add_node("writer", writer_agent)

workflow.set_entry_point("planner")
workflow.add_edge("planner", "retriever")
workflow.add_edge("retriever", "synthesizer")
workflow.add_edge("synthesizer", "reviewer")
workflow.add_edge("reviewer", "writer")
workflow.add_edge("writer", END)

app = workflow.compile()

print("--- Workflow compiled successfully. Ready to run. ---")

--- Assembling LangGraph workflow... ---
--- Workflow compiled successfully. Ready to run. ---


# EXECUTION AND FINAL OUTPUT
## This cell runs the compiled graph with a user-provided query.

In [8]:
print("--- Executing the workflow... ---")

query = input("Please enter your research query: ")
# Example query : 
# 1.Tell me the process of how a Machine learning model can differentiate between different patterns with the help of neural networks ?
# 2.How CNN is used for image processing ?
print("User Input :",query)

initial_state = {"original_query": query}

final_state = app.invoke(initial_state)

print("\n--- ✅ GRAPH EXECUTION COMPLETE ---")
print("Final Report:")
display(Markdown(final_state['final_report']))

--- Executing the workflow... ---
User Input : How CNN is used for image processing ?
--- 🧠 Executing Query Planner ---
--- 🔍 Executing Retriever (Strategy: both) ---
--- ✍️ Executing Synthesizer ---
--- ✅ Executing Reviewer ---
Reviewer LLM Response (Parsed): {'faithfulness_score': 0.95, 'review_notes': "The answer is mostly faithful, but the claim about 'standard benchmarks for deep learning algorithms in computer vision include object recognition and OCR' is not explicitly mentioned in the context. However, it can be inferred from the context that object recognition is a task that CNNs are applied to."}
--- 📄 Executing Writer ---

--- ✅ GRAPH EXECUTION COMPLETE ---
Final Report:


# Understanding CNNs in Image Processing

## Executive Summary
This report provides an overview of Convolutional Neural Networks (CNNs) and their application in image processing. It highlights the key mechanisms behind CNNs and their importance in computer vision tasks.

---

## Detailed Findings

### Convolutional Neural Networks (CNNs)

CNNs are a type of neural network that is widely applied to image data. They are designed to be invariant to certain transformations of the inputs, such as translation, rotation, and scaling. This is achieved through three mechanisms:

* Local receptive fields: Units in a feature map take inputs only from a small subregion of the image.
* Weight sharing: The same feature is computed across different regions of the image.
* Subsampling: Units in a feature map detect the same pattern but at different locations in the input image.

According to internal documentation, the structure of a convolutional network is illustrated in Figure 5.17, which shows a layer of convolutional units followed by a layer of subsampling units. Several successive pairs of such layers may be used.

### Parameter Sharing

Internal documentation highlights the concept of parameter sharing, where the same feature is computed across different regions of the image. This leads to large memory savings, as we need to store only a subset of the parameters. For example, in Convolutional Neural Networks or CNNs, the same feature is computed across different regions of the image, and hence, a cat is detected irrespective of whether it is at the top or bottom of the image.

### Computer Vision

Computer vision is one of the most active areas for deep learning research, as it is a task effortless for humans but difficult for computers. Standard benchmarks for deep learning algorithms in computer vision include object recognition and OCR (Optical Character Recognition). Computer vision requires little preprocessing, and images should be standardized, so pixels lie in the same range.

### Comparison of Sources

Upon comparing the internal documentation and web results, we notice that both sources agree on the importance of CNNs in image processing. However, there is a discrepancy in the level of detail provided. The internal documentation provides a more in-depth explanation of the mechanisms behind CNNs, while the web results provide a more general overview of the topic.

---

## Confidence & Sources

### Confidence Score
**95%**

### Sources
* Internal documentation: Provided a detailed explanation of the mechanisms behind CNNs, including local receptive fields, weight sharing, and subsampling.
* Web results: Provided a general overview of the importance of CNNs in image processing.

### References
* Internal documentation: Not publicly available.
* Web results: Various online sources, including [1] and [2].

[1] - [Insert reference 1]
[2] - [Insert reference 2]

Note: The references section should include all sources used in the report, including internal documentation and web results. However, since the internal documentation is not publicly available, it is not included in the references section.